# In class EM implementation

## Stopping iteration

Need to stop somewhere, keep track how much it is improving when it slows down then you stop optimizing.

How would actually calculate?

- Each sequences have is length $L$
- How to calculate things we need given the data?

## Computing posteriors (E step)

- Posteriors goes into M step
- At some point need to calculate post for each base for each sequence
    - Already init $\lambda_{l}$ and $\Psi_{l}$

- Could do nested for loop but not efficient

In current model each base is sampled independently so each time you see an A you will have same value.

$ P(C_{ij}=1|X_{ij}=A) = \frac{\lambda_{0}\Psi^{0}_{A}}{\lambda_{0}\Psi^{0}_{A} + \lambda_{1}\Psi^{1}_{A}} $

For next class write code to calculate the posteriors.